In [1]:
import os
import csv
import math
import random
import string
import functools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeRegressor

from keras.models import Sequential
from keras.layers import Dense

from objects.Plane import Plane
from objects.Maneuver import Mission_Maneuver, Maneuver_Mission
from objects.AllManeuvers import * 
from function import tools 
from function.tools import KNOTS2KMH, KMH2KNOTS


# WARNING : Values are wrong, this is just a sketch

# FUEL_CONSUMPTION_RATE =  3 #2.5 # KG / KM for a 100 knots speed for X plane model
pd.reset_option('display.max_colwidth')


Cet avion peut voler pendant  52438  s, soit  873  min, soit 2039  km à une vitesse moyenne de  140  km/h et sa vitesse max sera de  200  km/h


In [7]:

list_maneuvers = [
    ZigZag(180, 2000, 3, 30, 30),
    Spiral(180, 2000, 3, 30),
    Wheel(200, 2000, 5),
    Wheel(220, 2000, 5),
    Wheel(200, 1000, 5),
    Wheel(220, 1000, 5),
    Wheel(200, 1000, 5),
    Wheel(220, 1000, 5),
    Wheel(200, 2000, 5),
    Wheel(220, 2000, 5),
    Wheel(100, 20000, 5),
    # ShowOfForce('ClassicShowOfForce'),
]


In [8]:
for m in list_maneuvers:
    print(m.distance, m.travelled_time())

285.4115008234621 5708.23
191.9344667881098 3838.69
33.41592653589793 610.49
33.41592653589793 559.08
33.41592653589793 610.49
33.41592653589793 559.08
33.41592653589793 610.49
33.41592653589793 559.08
33.41592653589793 610.49
33.41592653589793 559.08
33.41592653589793 1175.97


In [5]:
dic_maneuvers = dict()

for i in range(len(list_maneuvers)):
    dic_maneuvers[i] = list_maneuvers[i]


In [ ]:
ULM = Plane()
M2000 = Plane('M2000', 0.2625, 3200, 532, 2340 * KMH2KNOTS)

plane_list = [M2000, ULM]

Cet avion peut voler pendant  52438  s, soit  873  min, soit 2039  km à une vitesse moyenne de  140  km/h et sa vitesse max sera de  200  km/h
Cet avion peut voler pendant  12190  s, soit  203  min, soit 1801  km à une vitesse moyenne de  532  km/h et sa vitesse max sera de  200  km/h


In [ ]:
print("Voila le coût en fuel des manoeuvres : ")
for plane in plane_list:
    print("Plane : ", plane)
    for man in list_maneuvers:
        print(man.total_fuel_consumption(plane),
              " kg for this maneuver | ", man.travelled_time(), " seconds for this maneuver, so ", math.floor(man.travelled_time() / 60), " min ", round(man.travelled_time() % 60, 2), " seconds")


Voila le coût en fuel des manoeuvres : 
Plane :  M2000
0  kg for this maneuver |  0  seconds for this maneuver, so  0  min  0  seconds
0  kg for this maneuver |  0  seconds for this maneuver, so  0  min  0  seconds
67.08229527611374  kg for this maneuver |  312.795918367347  seconds for this maneuver, so  5  min  12.8  seconds
67.12194817600985  kg for this maneuver |  308.70500927643786  seconds for this maneuver, so  5  min  8.71  seconds
67.18215379387331  kg for this maneuver |  312.795918367347  seconds for this maneuver, so  5  min  12.8  seconds
67.22191163990384  kg for this maneuver |  308.70500927643786  seconds for this maneuver, so  5  min  8.71  seconds
134.53410848421777  kg for this maneuver |  606.6734693877552  seconds for this maneuver, so  10  min  6.67  seconds
134.57386633024828  kg for this maneuver |  602.582560296846  seconds for this maneuver, so  10  min  2.58  seconds
134.34512247127128  kg for this maneuver |  606.6734693877552  seconds for this maneuver, so

In [ ]:
param_list = {
    'Plane': plane_list,
    'GoalDistance': [0, 10, 15, 20, 30],
    'RtBDistance': [0, 10, 15, 20, 30],
    'Fuel': [300, 400, 400, 450, 500, 550],
    # Start with just 2 states : good (sunny, no clouds) and bad (cloudy or rainy)
    'Meteo': ["Sunny", "Cloudy", "Misty"],
    'MissionType': [Mission_Maneuver.SCAR],  # , Mission_Maneuver.CAS
    # Add ennemies number afterwards, weaponry
    'Strength': ['Weak', 'Equal', 'Strong'],
    # '' : [90,0,50]
}

param_list = {
    'Plane': plane_list,  # Plane used in data
    # Distance from the starting point to the goal
    'GoalDistance': [0, 5, 10, 15, 20],
    # Distance from the objective to the exit point
    'RtBDistance': [0, 5, 10, 15, 20],
    # KG of fuel available at the start of the mission
    'Fuel': [300, 400, 400, 450, 500, 550],
    'Meteo': ["Sunny", "Cloudy", "Misty"],
    # Start with just 3 states : sunny (no clouds), cloudy and misty (cloudy++)
    'MissionType': [Mission_Maneuver.SCAR],
    # Type of mission : will change the mission context
    # , Mission_Maneuver.CAS
    # 'TimeMin': [None, 300],
    # Time constraint for the time on the objective
    # Add ennemies number afterwards, weaponry
    'Strength': ['Weak', 'Equal', 'Strong'],
    # 'Impredictibility' : [0, 50, 90], # Potential appearance of new ennemies on the objective point
}



In [ ]:
all_combinations_not_filtered = tools.get_all_combinations(param_list)
all_combinations = []
for data in all_combinations_not_filtered:

    plane = data['Plane']
    if plane.name == 'M2000':
        data['Fuel'] *= 6
    elif plane.name == 'ULM':
        data['Fuel'] = round(data['Fuel'] / 6)
    # Check if the data corresponds to the plane caracteristics
    if plane.fuel_max >= data['Fuel']:
        all_combinations.append(data)


In [ ]:
print(len(all_combinations))
print(len(all_combinations_not_filtered))
print(pd.DataFrame(all_combinations).sample(10))

2475
2700
      Plane  GoalDistance  RtBDistance  Fuel   Meteo MissionType Strength
2212    ULM            20            0    50   Misty        SCAR    Equal
427   M2000             5           20  2400  Cloudy        SCAR    Equal
1442    ULM             5            0    92   Sunny        SCAR   Strong
1140    ULM             0            0    67   Misty        SCAR     Weak
167   M2000             0           15  2700  Cloudy        SCAR   Strong
2218    ULM            20            0    67  Cloudy        SCAR    Equal
220   M2000             0           20  3000  Cloudy        SCAR    Equal
247   M2000             5            0  2400  Cloudy        SCAR    Equal
1105  M2000            20           20  2400   Misty        SCAR    Equal
2261    ULM            20            5    50   Sunny        SCAR   Strong


In [ ]:
for c in all_combinations:
    plane = c['Plane']
    fcr = plane.fuel_consumption_rate
    speed = c['MaxSpeedOP'] * KNOTS2KMH
    c['FuelRemaining'] = c['Fuel'] - (fcr * (c['GoalDistance'] / speed) * 3600) - (
        fcr * (c['RtBDistance'] / speed) * 3600)
    # c['BaseFuelRemaining'] = c['FuelRemaining']


KeyError: 'MaxSpeedOP'

In [ ]:
def choose_randomly(maneuvers_for_c, c) -> list:
    percent = math.ceil(random.random() * 100)
    # percent = math.ceil(random.gauss(50,(50-5)/3))
    if percent < 0:
        percent += 100
    elif percent > 100:
        percent -= 100
    max_speed = c['MaxSpeedOP']
    new_maneuvers = []
    new_maneuvers = filter(
        lambda m: dic_maneuvers[m].meanspeed <= max_speed, maneuvers_for_c)
    if c['Strength'] == 'Weak' or c['Strength'] == 'Equal':
        if percent <= 95:
            new_maneuvers = filter(
                lambda m: dic_maneuvers[m].meanspeed >= 200, new_maneuvers)
        else:
            new_maneuvers = filter(
                lambda m: dic_maneuvers[m].meanspeed < 200, new_maneuvers)
    elif c['Strength'] == 'Strong':
        if percent <= 95:
            new_maneuvers = filter(
                lambda m: dic_maneuvers[m].meanspeed < 200, new_maneuvers)
        else:
            new_maneuvers = filter(
                lambda m: dic_maneuvers[m].meanspeed >= 200, new_maneuvers)

    if c['Meteo'] == "Cloudy" or c['Meteo'] == 'Misty':
        if percent <= 95:
            new_maneuvers = filter(
                lambda m: dic_maneuvers[m].altitude < 1, new_maneuvers)
        else:
            new_maneuvers = filter(
                lambda m: dic_maneuvers[m].altitude >= 1, new_maneuvers)
    else:
        if percent <= 95:
            new_maneuvers = filter(
                lambda m: dic_maneuvers[m].altitude >= 1, new_maneuvers)
        else:
            new_maneuvers = filter(
                lambda m: dic_maneuvers[m].altitude < 1, new_maneuvers)
    return list(new_maneuvers)


In [ ]:
def select_maneuvers(maneuvers_for_c, c):
    selected = []
    minCost = min(dic_maneuvers[man].total_fuel_consumption(
        c['Plane']) for man in maneuvers_for_c)
    minEfficiency = min(
        dic_maneuvers[man].efficiency for man in maneuvers_for_c)
    efficiency = 90
    while c['FuelRemaining'] >= minCost and efficiency >= minEfficiency and len(selected) < 5:
        maneuvers_with_eff = list(filter(
            lambda man: dic_maneuvers[man].efficiency >= efficiency, maneuvers_for_c))
        if len(maneuvers_with_eff) != 0:
            rand = random.randint(0, len(maneuvers_with_eff)-1)
            man = dic_maneuvers[maneuvers_with_eff[rand]]
            if (c['FuelRemaining'] - man.total_fuel_consumption(c['Plane'])) >= 0:
                c['FuelRemaining'] -= man.total_fuel_consumption(c['Plane'])
                selected.append(maneuvers_with_eff[rand])
            else:
                efficiency -= 5
        else:
            efficiency -= 5
    return selected


In [ ]:
indexed_maneuvers = list_maneuvers.copy()

p = True
combinations = all_combinations.copy()
count = 0
all_maneuvers_for_c = []
for c in combinations:
    maneuvers_for_c = list(filter(
        lambda m:  dic_maneuvers[m].name in c['MissionType'].getManeuver(), dic_maneuvers))
    maneuvers_for_c = list(filter(lambda m: dic_maneuvers[m].total_fuel_consumption(
        c['Plane']) < c['FuelRemaining'], maneuvers_for_c))
    maneuvers_for_c = choose_randomly(maneuvers_for_c, c)
    all_maneuvers_for_c.append(maneuvers_for_c)


In [ ]:
final_combinations = combinations.copy()
for c, maneuvers_for_c in zip(final_combinations, all_maneuvers_for_c):
    if len(maneuvers_for_c) > 0:
        maneuvers = select_maneuvers(maneuvers_for_c, c)
        if maneuvers != None or len(maneuvers) > 0:
            c['Maneuvers'] = list(maneuvers)
            c['TimeOfMission'] = sum(
                dic_maneuvers[m].travelled_time() for m in c['Maneuvers'])


In [ ]:
# print(impossible_combinations)
final_combinations_plane = [
    c for c in final_combinations if c.get("Maneuvers")]
# print(len(final_combinations), len(final_combinations_plane))
# for c in final_combinations_plane:
#     # c['Maneuvers'] = [c.__dict__ for c in c['Maneuvers']]
#     # c['Plane'] = c['Plane']

final_combinations_maneuvers = final_combinations_plane.copy()
# final_combinations_maneuvers = final_combinations.copy()
list_maneuvers = []

for c in final_combinations_maneuvers:
    list_maneuvers.append(c['Maneuvers'])

print(len(list_maneuvers))
first_tuple_list = []
for lst in list_maneuvers:
    first_tuple_list.append(tuple(lst))
print("Done")
set_man = set(first_tuple_list)
new_dict = dict()

for i in range(len(set_man)):
    new_dict[set_man.pop()] = i
print(len(new_dict))


0
Done
0


In [ ]:
# print(dic_maneuvers.get(i))

for c in final_combinations_maneuvers:
    c['Maneuvers'] = new_dict.get(tuple(c['Maneuvers']))
df = pd.DataFrame(final_combinations_maneuvers)


In [ ]:

df.describe()


ValueError: Cannot describe a DataFrame without columns

In [ ]:
pd.set_option('display.max_colwidth', None)
df.sample(1).get("Maneuvers")
pd.reset_option('display.max_colwidth')


In [ ]:


df = pd.get_dummies(df, columns=["Plane", "Meteo", "MissionType", "Strength", 'TimeMin'])
df.sample(10)
# data_MissionType = pd.DataFrame(ohe.fit_transform(data_meteo[["MissionType"]]))
# data_Strength = pd.DataFrame(ohe.fit_transform(data_MissionType[["Strength"]]))
# data_Strength


In [ ]:
df_first = df.copy()
X_train, X_test, y_train, y_test = train_test_split(df_first.drop(
    "TimeOfMission", axis=1), df_first["TimeOfMission"], test_size=0.15)
# X_train, X_test, y_train, y_test = train_test_split( df.drop("Maneuvers", axis=1), df["Maneuvers"], test_size=0.30)
models = [Lasso(), KNeighborsRegressor(), RandomForestRegressor(),
          GradientBoostingRegressor()]

once = True
for model in models:

    # Model is trained with the corrupted / imputed data
    model.fit(X_train, y_train)

    # Model performance is evaluated against the original test dataset
    y_pred = model.predict(X_test)
    # y_pred = [round(y) for y in y_pred]
    score = r2_score(y_test, y_pred)
    if not once:
        for t, p in zip(y_test, y_pred):
            print(t, p)
        once = True
    print(model, score)


In [ ]:
tools.showMatrix(df_first)

In [ ]:

df_second = df.copy()
# , df_second["Maneuvers"],]
X_train, X_test, y_train, y_test = train_test_split(df_second.drop(
    ["TimeOfMission", "Maneuvers", "FuelRemaining"], axis=1), df_second["TimeOfMission"], test_size=0.30)
# X_train, X_test, y_train, y_test = train_test_split( df.drop("Maneuvers", axis=1), df["Maneuvers"], test_size=0.30)
models = [Lasso(), KNeighborsRegressor(), RandomForestRegressor(
    n_estimators=100), GradientBoostingRegressor(), DecisionTreeRegressor(max_depth=5)]

print(X_train.columns)


In [ ]:

for model in models:
    # Model is trained with the corrupted / imputed data
    model.fit(X_train, y_train)

once = True
dim = X_train.shape[1]
epochs = 50
activation_names = ["relu", "sigmoid", "tanh"]
# for activation in activation_names :
#     model = Sequential()
#     #On pourrait essayer de rajouter plusieurs paramètres de modèles différents notamment en rajoutant des couches supplémentaires ou en utilisant un nombre de neurones cachés différent. Nous allons nous contenter ici de seulement changer la fonction d'activation.
#     model.add(Dense(150, input_dim= dim, activation=activation))
#     #On peut essayer d'ajouter une couche intermédiaire, mais la performance ne sera pas assez intéressante
#     #model.add(Dense(300, activation=name))
#     model.add(Dense(150, activation=activation))

#     # Vu que l'on utilise une régression pour un prix, on utilise une fonction d'activation linéaire pour la sortie
#     model.add(Dense(1, activation='linear'))
#     model.compile(loss='mean_squared_error', optimizer='adam')

#     #visualize_model(X_train, X_test, y_train, y_test, model, show_pred_vs_y_test=True)
#     model.fit(X_train, y_train,epochs=epochs, verbose = 0)

#     models.append(model)
scores = []
for model in models:

    # Model performance is evaluated against the original test dataset
    y_pred = model.predict(X_test)
    # y_pred = [round(y) for y in y_pred]
    scores.append(r2_score(y_test, y_pred))
    if not once:
        for t, p, x in zip(y_test, y_pred, X_test.values):
            print(t, p, x)
        once = True


In [ ]:
for model, score in zip(models, scores):
    print(model, score)


In [ ]:
tools.showMatrix(df_second)



In [ ]:
# Export as dot file
# export_graphviz(models[4], out_file='tree.dot',
#                 feature_names=df_second.drop(
#                     ["TimeOfMission", "FuelRemaining", "Maneuvers"], axis=1).columns,
#                 class_names="TimeOfMission",
#                 rounded=True, proportion=False,
#                 precision=6, filled=True)
# os.system('dot -Tpng tree.dot -o tree.png')
# Display in jupyter notebook


In [ ]:

tools.showMatrix(df_second.drop(
    ["Maneuvers", "FuelRemaining"], axis=1))

In [ ]:
col_names = []
nb_params = 1
for i in range(nb_params):
    col_names.append("Y{}".format(i+1))
df_y = pd.DataFrame(K.zeros((df.shape[0], nb_params)), columns=col_names)

print(df.shape, df_y.shape)

# df = np.expand_dims(df, -1)
# df_y = np.expand_dims(df_y, -1)
X_train, X_test, y_train, y_test = train_test_split(df, df_y, test_size=0.30)
X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
print(y_test.shape)